## Sentiment Analysis with Camembert

### Load dataset from tblard/allocine

In [1]:
from datasets import load_dataset

ds = load_dataset("tblard/allocine")

README.md:   0%|          | 0.00/9.31k [00:00<?, ?B/s]

c:\Users\visalk\.conda\envs\email_classification\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\visalk\.cache\huggingface\hub\datasets--tblard--allocine. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/60.0M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/7.58M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/7.58M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [30]:
ds = ds[:1000]

KeyError: "Invalid key: slice(None, 1000, None). Please first select a split. For example: `my_dataset_dictionary['train'][slice(None, 1000, None)]`. Available splits: ['test', 'train', 'validation']"

In [14]:
ds["train"]

Dataset({
    features: ['review', 'label'],
    num_rows: 160000
})

In [37]:
import pandas as pd

data_train = ds["train"].select(range(3000))
data_validation = ds["validation"].select(range(3000))
data_test = ds["test"].select(range(3000))

In [41]:
ds

DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 160000
    })
    validation: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 20000
    })
})

### Model selection and experimentaion

In [18]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

classes = [0, 1]
tokenizer = AutoTokenizer.from_pretrained("almanach/camembert-base")
model = AutoModelForSequenceClassification.from_pretrained("almanach/camembert-base", num_labels = len(classes))


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\visalk\.conda\envs\email_classification\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\visalk\.cache\huggingface\hub\models--almanach--camembert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at almanach/camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Test model

In [25]:
data_train["review"][4]

'Premier film de la saga Kozure Okami, "Le Sabre de la vengeance" est un très bon film qui mêle drame et action, et qui, en 40 ans, n\'a pas pris une ride.'

In [26]:
input_string = data_train["review"][4]
inputs = tokenizer(
    input_string,
    return_tensors="pt",  # PyTorch tensors
    truncation=True,      # Truncate if the text is too long
    padding=True,         # Add padding if needed
    max_length=256        # Ensure input length fits the model
)

model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).item() 

print(predictions)

0


Data preprocessing

In [27]:
print(type(data_train))

<class 'datasets.arrow_dataset.Dataset'>


In [39]:
def tokenize(batch):
    return tokenizer(
        batch["review"], 
        padding="max_length", 
        truncation=True, 
        max_length=256  # Adjust max_length to suit your model
    )

model.train()
tokenized_train = data_train.map(tokenize, batched=True)
tokenized_validation = data_validation.map(tokenize, batched=True)
tokenized_test = data_test.map(tokenize, batched=True)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
import wandb
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import numpy as np

wandb.login()
wandb.init(project="Frech-sentiment-analysis", name="fine_tune_camembert-base")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, np.ndarray):
        logits = torch.tensor(logits)
    if isinstance(labels, np.ndarray):
        labels = torch.tensor(labels)
    predictions = torch.argmax(logits, dim=-1)
    accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
    return {"accuracy": accuracy}

training_arguments = TrainingArguments(
    output_dir='/.results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="wandb",  # Enable wandb tracking
    logging_dir="./logs",  # Log directory
    logging_steps=10,      # Adjust logging frequency
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    compute_metrics=compute_metrics,
)

trainer.train()

model.save_pretrained("finetuned_model")
tokenizer.save_pretrained("finetuned_model")

wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
c:\Users\visalk\.conda\envs\email_classification\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.130500,0.264722,0.943000
